In [2]:
from selenium import webdriver
import requests
import os

In [4]:
# Creamos una función que extraiga robots.txt

def extraer_robots(url):
    if url.endswith('/'):
        direc = url
    else:
        direc = url + '/'
    extrac = requests.get(direc+'robots.txt',data=None)
    return extrac.text
 

In [ ]:
# La URL va a ser la página sullygnome donde están los datos buscados

URL = 'https://sullygnome.com/'

In [ ]:

# Leemos el robots.txt
print(extraer_robots(URL))

# Opciones del webdriver
option = webdriver.ChromeOptions()
option.add_argument(" — incognito") #open incognito mode
option.add_argument("user-agent=AcademicCrawler")#set our UserAgent name, in this case AcademicCrawler


# Buscamos el directorio de la carpeta
My_path = os.path.dirname(os.path.abspath(__file__))

# Proponemos 5s de delay
TimeOut = 5 #sec

#Buscamos el driver de Chrome (Download from http://chromedriver.chromium.org/downloads)
browser = webdriver.Chrome(executable_path=My_path + '/chromedriver', chrome_options=option)

# Get content from objective website
browser.get("https://sullygnome.com/channels/365/mostfollowers")

# Aplicamos el delay
browser.implicitly_wait(TimeOut);



# Buscamos los url de los 50 top streamers
elements = browser.find_elements_by_xpath("""//*[@id="tblControl"]/tbody/tr/td[1]/a""")
links = []
for element in elements:
    links.append(element.get_attribute("href"))
    

# Introducimos los links en un diccionario
dictlist = []
for link in links:
    browser.get(link)
    browser.implicitly_wait(TimeOut);
    userStatsDict={}
    
# ----- A PARTIR DE AQUÍ DEJA DE FUNCIONAR (no encuentro la manera adecuada de extraer estos datos)------
# Scraping
followers = browser.find_elements_by_css_selector("body > div.RightContent > div.MainContent > div.PageContentContainer > div.StandardPageContainer > div > div:nth-child(1) > div:nth-child(1) > div > div.PageBackgroundColor.InfoPanelMiddle > div:nth-child(3) > div.InfoPanelRowRight.BoldText")
streams = browser.find_elements_by_css_selector("body > div.RightContent > div.MainContent > div.PageContentContainer > div.StandardPageContainer > div > div:nth-child(1) > div:nth-child(1) > div > div.PageBackgroundColor.InfoPanelMiddle > div:nth-child(5) > div.InfoPanelRowRight.BoldText > span:nth-child(1)")
averageViews = browser.find_elements_by_css_selector("body > div.RightContent > div.MainContent > div.PageContentContainer > div.StandardPageContainer > div > div:nth-child(1) > div:nth-child(2) > div > div.PageBackgroundColor.InfoPanelMiddle > div:nth-child(1) > div.InfoPanelRowRight.BoldText")
watchTime = browser.find_elements_by_css_selector("body > div.RightContent > div.MainContent > div.PageContentContainer > div.StandardPageContainer > div > div:nth-child(1) > div:nth-child(2) > div > div.PageBackgroundColor.InfoPanelMiddle > div:nth-child(3) > div.InfoPanelRowRight.BoldText")
peakViewers = browser.find_elements_by_css_selector("body > div.RightContent > div.MainContent > div.PageContentContainer > div.StandardPageContainer > div > div:nth-child(1) > div:nth-child(2) > div > div.PageBackgroundColor.InfoPanelMiddle > div:nth-child(5) > div.InfoPanelRowRight.BoldText")
streamTime = browser.find_elements_by_css_selector("body > div.RightContent > div.MainContent > div.PageContentContainer > div.StandardPageContainer > div > div:nth-child(1) > div:nth-child(2) > div > div.PageBackgroundColor.InfoPanelMiddle > div:nth-child(7) > div.InfoPanelRowRight.BoldText")
topGame = browser.find_elements_by_xpath("""//*[@id="combinedPanel"]/div/div[2]/div/div[2]/div[8]/div[1]/a""")


# Guardamos variables en diccionario
userStatsDict['User']=link
userStatsDict['Followers']=followers[0].text
userStatsDict['Streams']=streams[0].text
userStatsDict['AverageViews']=averageViews[0].text
userStatsDict['WatchTime']=watchTime[0].text
userStatsDict['PeakViewers']=peakViewers[0].text
userStatsDict['StreamTime']=streamTime[0].text
userStatsDict['topGameLink']=topGame[0].get_attribute("href")

dictlist.append(userStatsDict)

# Crear y/o sobreescribir los datos

filename = "/TwitchData.csv"
file = open(My_path + filename, "w+")
#Get the keys of the dictionary
keys = []
for key in userStatsDict:
    keys.append(key)
    

    
# Eliminamos los datos de csv
for i in range(len(keys)):
    file.write(str(keys[i]) + ";");
    file.write("\n");
    
for i in range(len(dictlist)):
    for j in range(len(keys)):
        file.write(str(dictlist[i][keys[j]]) + ";");
    file.write("\n");
    
file.close()
browser.quit()